# MMA基金資料

In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd

In [2]:
import re
from collections import defaultdict

In [143]:
requests.packages.urllib3.disable_warnings()  # disable warning insecure

______

## 國內

### 基本資料


In [566]:
uid = 'ACUS03-UN2'
url_basic = 'http://mmafund.sinopac.com/w/wr/wr01.djhtm?a=ACUS03-UN2'
res = requests.get(url_basic)

In [570]:
soup = BS(res.text,"lxml")

* 基本資料

In [571]:
# r"(?:a=)(.+)"
fundid = re.findall(r'(?:a=)(.+)',soup.select('#itemTab')[0].find('a').get('href'))[0]
fundid

'ACUS03-UN2'

In [572]:
fieldSoup = soup.select('.wfb5l')
fieldList = [uid]
for e in fieldSoup[:10]:
#     print(e.text)
    fieldList.append(e.text)
starNum = getFundStar(fieldSoup[-3])
fieldList.append(starNum)
fieldList.append(fieldSoup[-1].text)
fieldList

星星幾顆:4.5


['ACUS03-UN2',
 '永豐領航科技基金',
 '永豐投信',
 '1998年9月4日',
 '楊明昌',
 '6.43台幣 (2017/01/31)',
 '2.00(台幣)',
 '國內股票開放型科技類',
 '中國信託銀行',
 '台灣',
 '台灣',
 4.5,
 '本基金以投資從事或轉投資於通訊、資訊、半導體、關鍵性零組件、生物工程、消費性電子、精密機械與自動化、航太、高級材料、特用化學與製藥、醫療保健、污染防治、資源開發、高級感測及其他經財政部會同主管機關認定重要科技之上市、上櫃股票為主，投資總額不低於淨資產之60%']

In [573]:
info_colName = [
    'uid',
    '基金名稱',
    '基金公司',
    '成立日期',
    '基金經理人',
    '基金規模(億)',
    '成立時規模(億)',
    '基金類型',
    '保管銀行',
    '主要投資區域',
    '投資區域',
    '基金評等',
    '投資標的'
]
list(zip(info_colName,fieldList))

[('uid', 'ACUS03-UN2'),
 ('基金名稱', '永豐領航科技基金'),
 ('基金公司', '永豐投信'),
 ('成立日期', '1998年9月4日'),
 ('基金經理人', '楊明昌'),
 ('基金規模(億)', '6.43台幣 (2017/01/31)'),
 ('成立時規模(億)', '2.00(台幣)'),
 ('基金類型', '國內股票開放型科技類'),
 ('保管銀行', '中國信託銀行'),
 ('主要投資區域', '台灣'),
 ('投資區域', '台灣'),
 ('基金評等', 4.5),
 ('投資標的',
  '本基金以投資從事或轉投資於通訊、資訊、半導體、關鍵性零組件、生物工程、消費性電子、精密機械與自動化、航太、高級材料、特用化學與製藥、醫療保健、污染防治、資源開發、高級感測及其他經財政部會同主管機關認定重要科技之上市、上櫃股票為主，投資總額不低於淨資產之60%')]

* 找星星幾枚?

In [574]:
starList = fieldSoup[-3].select('img') # bs4 tag
starList = [star['src'] for star in starList ] # 取出src裡面的url
starStr = '**'.join(starList) # 接成string
# starList[0]['src']
if re.findall('/BT_Coin_b.gif',starStr):
    starNum = 0.5
else:
    starNum = 0
starNum += len(re.findall('/BT_Coin_a.gif',starStr))
print('星星幾顆:{}'.format(starNum))
# starStr

星星幾顆:4.5


In [575]:
getFundStar(fieldSoup[-3])

星星幾顆:4.5


4.5

#### 歷任經理人

In [576]:
managerList = []

managerBSTable = soup.select('#oMainTable')[1]


managerList = []
for manager in managerBSTable.select('tr')[2:]:
    row = []
    for index,field in enumerate(manager.select('td')):
        try:
            row.append(float(field.text)) # 轉換數字部分
        except ValueError as e:
            row.append(field.text) 
    print(row)

['楊明昌', '2017/2/3至今', 1.0, 4.21, 3.41, '永豐高科技基金\n永豐領航科技基金']
['林靜怡', '2014/08/01至2017/2/2', 30.0, 27.23, 1.21, '']
['李盈儀', '2014/05/02至2014/7/31', 2.0, 7.63, 5.97, '永豐中小基金\n永豐永豐基金']
['邱鶴倫', '2012/04/01至2014/5/1', 25.0, 46.81, 10.82, '']
['謝其勛', '2011/04/29至2012/3/31', 11.0, -22.13, -12.25, '']
['李維岳', '2010/04/20至2011/4/28', 12.0, 6.0, 15.11, '']
['謝其勛', '2007/11/05至2010/4/19', 29.0, -27.62, -15.3, '']
['張家維', '2003/01/02至2007/11/4', 58.0, 201.6, 108.27, '']
['謝宗權', '2002/03/20至2003/1/1', 10.0, -41.2, -26.52, '']
['葉順吉', '2001/03/01至2002/3/19', 12.0, 5.08, 7.4, '']
['林大鈞', '1998/09/01至2001/2/28', 29.0, 21.6, -16.56, '富蘭克林華美積極回報債券組合基金-累積型(台幣)(本基金有相當比重投資於非投資等級之高風險債券且基金之配息來源可能為本金)\n富蘭克林華美積極回報債券組合基金-分配型(台幣)(本基金有相當比重投資於非投資等級之高風險債券且基金之配息來源可能為本金)\n富蘭克林華美積極回報債券組合基金-累積型(美元)(本基金有相當比重投資於非投資等級之高風險債券且基金之配息來源可能為本金)\n富蘭克林華美積極回報債券組合基金-分配型(人民幣)(本基金有相當比重投資於非投資等級之高風險債券且基金之配息來源可能為本金)\n富蘭克林華美全球投資級債券基金-分配型(台幣)(本基金之配息來源可能為本金)\n富蘭克林華美積極回報債券組合基金-分配型(美元)(本基金有相當比重投資於非投資等級之高風險債券且基金之配息來源可能為本金)\n富蘭克林華美全球投資級債券基金-分配型(

### 持股狀況

In [577]:
url_fund = 'http://mmafund.sinopac.com/w/wr/wr04.djhtm?a=ACJS24-Z21'
res =requests.get(url_fund)

In [578]:
soup = BS(res.text,"lxml")

In [579]:
update_dateStr = soup.select('#oMainTable')[1].select('tr')[0].text # 資料月份
update_date = re.findall(r'\d+/\d+',update_dateStr)[0]
print(update_date)

2017/01


In [580]:
for e in soup.select('#oMainTable')[1].select('tr')[1:]:
    row1 = []; row2=[];
    for index,field in enumerate(e.select('td')):
        if index > 3:
            row2.append(field.text)
        else:
            row1.append(field.text)
    print('row1:%s \nrow2:%s' %(row1,row2))

row1:['股票名稱', '持股(千股)', '比例', '增減'] 
row2:['股票名稱', '持股(千股)', '比例', '增減']
row1:['Tencent', 'N/A', '6.65', 'N/A'] 
row2:['KOITO MFG CO', 'N/A', '3.07', '-0.14%']
row1:['大立光', '6', '6.61', '4.36%'] 
row2:['華夏滬深300指數ETF', 'N/A', '2.36', '0.00%']
row1:['台積電', '130', '5.97', '0.05%'] 
row2:['Telekomunikasi Indonesia TBK', 'N/A', '2.25', '-0.15%']
row1:['NIDEC CORP', 'N/A', '3.63', '0.14%'] 
row2:['正新', '145', '2.24', '0.89%']
row1:['環球晶', '90', '3.19', '-3.25%'] 
row2:['iShares 安碩富時A50 中國指數ETF', 'N/A', '2.23', '0.03%']


### 基金持股分布

* 依持有類股
* 依產業
* 依區域

資料藏在js語法裏頭

In [581]:
soup.select('#oMainTable')[2].select('div')

[<div class="wfb1ar">資料日期：2017/02/17</div>,
 <div id="SysJustWebGraphDIV1"></div>]

In [582]:
string1 = 'DJGraphObj1'
html_text = res.text
target_text = html_text[html_text.index(string1):]
# print(html_text[html_text.index(string1):])
print(target_text)

DJGraphObj1;
	var $DJGraphObj2;
	var $DJGraphObj3;
	
	var sRenderTo1 = 'SysJustWebGraphDIV1';
	var sRenderTo2 = 'SysJustWebGraphDIV2';
	var sRenderTo3 = 'SysJustWebGraphDIV3';
	
	$(document).ready(function () {
		if (!!$DJWebGraph) {

			$DJGraphObj1 = new $DJWebGraph();
	        $DJGraphObj2 = new $DJWebGraph();
			$DJGraphObj3 = new $DJWebGraph();
        	
			if (sRenderTo1 != '')
			{
				$DJGraphObj1.CreateGraphObject({
					'CanvasClass': 'pie', //表示為 PIE 圖形
					'ID': 'wb04-1',
					'Width': '500',
					'Height': '300',
					'RenderTo': sRenderTo1,
					'Title': ['', '基金持股分佈-依持有類股[萬元]','亞洲不含日本','日本','大陸地區','上市光電','上市半導體','上櫃半導體','上市橡膠','上市電子零組件','上市油電燃氣','上櫃其他電子','上市電腦及週邊設備','其它'],
					'PieBorder' : '0',
					'BgColor': 'ffffff',
					'PieNOTB': 0,		//0:顯示表格,1:不顯示表格,2:不顯示表格,只顯示名稱
					'PieSplit': ['1','0','0','0','0','0','0','0','0','0','0','0'],
					'PieV': ['16416','4583','3558','2681','2431','1619','937','892','844','440','279','5702'],
					'Pie_AddHeaders': '名稱 投資金額(以萬元為單

In [584]:
pat1 = r"(?:\'Title\':)(.+)"
investTitle = re.findall(pat1,target_text)
print(investTitle[0])

 ['', '基金持股分佈-依持有類股[萬元]','亞洲不含日本','日本','大陸地區','上市光電','上市半導體','上櫃半導體','上市橡膠','上市電子零組件','上市油電燃氣','上櫃其他電子','上市電腦及週邊設備','其它'],


In [585]:
pat2 = r"(?:\')(.*?)(?:\')"
investTitleByStock = re.findall(pat2,investTitle[0]) ## 依產業標題
investTitleByStock

['',
 '基金持股分佈-依持有類股[萬元]',
 '亞洲不含日本',
 '日本',
 '大陸地區',
 '上市光電',
 '上市半導體',
 '上櫃半導體',
 '上市橡膠',
 '上市電子零組件',
 '上市油電燃氣',
 '上櫃其他電子',
 '上市電腦及週邊設備',
 '其它']

In [586]:
investTitleByArea = re.findall(pat2,investTitle[1])
investTitleByArea

['', '基金持股分佈-依區域[萬元]', '香港', '台灣', '中國', '日本', '現金', '東南亞', '韓國']

In [587]:
investTitleByIndustry = re.findall(pat2,investTitle[2])
investTitleByIndustry

['',
 '基金持股分佈-依產業[萬元]',
 '科技',
 '工業',
 '消費',
 '金融保險',
 '現金',
 'A股ETF',
 '能源',
 '原物料',
 '公共事業',
 '醫療保健',
 '其他']

In [588]:
pat3 = r"(?:\'PieV\':)(.+)"
investByStock = re.findall(pat2,re.findall(pat3,target_text)[0])
investByArea = re.findall(pat2,re.findall(pat3,target_text)[1])
investByIndustry = re.findall(pat2,re.findall(pat3,target_text)[2])

## 境外

### 基金基本資料

In [617]:
url_wbTest_info = 'http://mmafund.sinopac.com/w/wb/wb01.djhtm?a=ANZ11-A06'
html_text_wb = requests.get(url_wbTest_info).text
soup = BS(html_text_wb,"lxml")

In [592]:
fieldSoup = soup.select('.wfb5l')
fieldList = []
for e in fieldSoup[:15]:
#     print(e.text)
    fieldList.append(e.text)
starNum = getFundStar(fieldSoup[-4])
fieldList.append(starNum)
fieldList.append(fieldSoup[-2].text)
fieldList

星星幾顆:3


['安本環球英國股票基金',
 'Aberdeen Global-UK Equity Fund A2',
 'Aberdeen安本投顧',
 'Aberdeen安本資產管理',
 'FTSE ALL Share',
 '1988/04/26',
 '盧森堡',
 '37 百萬英鎊(2016/12/31)',
 '英鎊',
 '單一國家型基金',
 '英國',
 '股票型',
 '法國巴黎銀行盧森堡分行',
 'Y',
 '泛歐洲股票團隊',
 3,
 '本基金的投資目標是通過將該項基金至少三分之二的資產，投資於在英國設有註冊辦事處的公司；及/ 或在英國進行大部份業務活動的公司；及/ 或其大部份資產來自在英國設有註冊辦事處的公司的控股公司所發行的股票及股票相關證券。該項基金的基本貨幣是英鎊。']

In [399]:
titleList = []
for title in soup.select('.wfb3c'):
    print(title.text)
    titleList.append(title.text)
index = [-1,-2]
for popele in index :
    titleList.pop(popele)
len(titleList)

基金名稱
基金英文名稱
台灣總代理
海外發行公司
指標指數
成立日期
註冊地
基金規模
計價幣別
基金類型
投資區域
投資標的
保管機構
傘狀基金
經理人
基金評等
文件下載
投資策略
投資人服務及保護


17

In [408]:
colNames = [
    '基金名稱',
    '基金英文名稱',
    '台灣總代理',
    '海外發行公司',
    '指標指數',
    '成立日期',
    '註冊地',
    '基金規模',
    '計價幣別',
    '基金類型',
    '投資區域',
    '投資標的',
    '保管機構',
    '傘狀基金',
    '經理人',
    '基金評等',    
    '投資策略'    
]

In [411]:
list(zip(colNames,fieldList))

[('基金名稱', '安本環球英國股票基金'),
 ('基金英文名稱', 'Aberdeen Global-UK Equity Fund A2'),
 ('台灣總代理', 'Aberdeen安本投顧'),
 ('海外發行公司', 'Aberdeen安本資產管理'),
 ('指標指數', 'FTSE ALL Share'),
 ('成立日期', '1988/04/26'),
 ('註冊地', '盧森堡'),
 ('基金規模', '37 百萬英鎊(2016/12/31)'),
 ('計價幣別', '英鎊'),
 ('基金類型', '單一國家型基金'),
 ('投資區域', '英國'),
 ('投資標的', '股票型'),
 ('保管機構', '法國巴黎銀行盧森堡分行'),
 ('傘狀基金', 'Y'),
 ('經理人', '泛歐洲股票團隊'),
 ('基金評等', 3),
 ('投資策略',
  '本基金的投資目標是通過將該項基金至少三分之二的資產，投資於在英國設有註冊辦事處的公司；及/ 或在英國進行大部份業務活動的公司；及/ 或其大部份資產來自在英國設有註冊辦事處的公司的控股公司所發行的股票及股票相關證券。該項基金的基本貨幣是英鎊。')]

In [620]:
def getForeignFundInfo(soup):
    """
    """
    ## fundid ##
    fundid = re.findall(r"(?:a=)(.+)",soup.select('#itemTab')[0].find('a').get('href'))[0]
    
    fieldSoup = soup.select('.wfb5l')
    fieldList = []
    for e in fieldSoup[:15]:
    #     print(e.text)
        fieldList.append(e.text)
    starNum = getFundStar(fieldSoup[-4])
    fieldList.append(starNum)
    fieldList.append(fieldSoup[-2].text)
    colNames = [
        '基金名稱',
        '基金英文名稱',
        '台灣總代理',
        '海外發行公司',
        '指標指數',
        '成立日期',
        '註冊地',
        '基金規模',
        '計價幣別',
        '基金類型',
        '投資區域',
        '投資標的',
        '保管機構',
        '傘狀基金',
        '經理人',
        '基金評等',    
        '投資策略'    
    ]
    dataList = list(zip(colNames,fieldList))
    fundInfo_dict = defaultdict(list)
    fundInfo_dict['fundid'] = fundid
    for name,v in dataList:
        fundInfo_dict[name] = v
    return fundInfo_dict

In [621]:
getForeignFundInfo(soup)

星星幾顆:3


defaultdict(list,
            {'fundid': 'ANZ11-A06',
             '保管機構': '法國巴黎銀行盧森堡分行',
             '傘狀基金': 'Y',
             '台灣總代理': 'Aberdeen安本投顧',
             '基金名稱': '安本環球英國股票基金',
             '基金英文名稱': 'Aberdeen Global-UK Equity Fund A2',
             '基金規模': '37 百萬英鎊(2016/12/31)',
             '基金評等': 3,
             '基金類型': '單一國家型基金',
             '成立日期': '1988/04/26',
             '投資區域': '英國',
             '投資標的': '股票型',
             '投資策略': '本基金的投資目標是通過將該項基金至少三分之二的資產，投資於在英國設有註冊辦事處的公司；及/ 或在英國進行大部份業務活動的公司；及/ 或其大部份資產來自在英國設有註冊辦事處的公司的控股公司所發行的股票及股票相關證券。該項基金的基本貨幣是英鎊。',
             '指標指數': 'FTSE ALL Share',
             '海外發行公司': 'Aberdeen安本資產管理',
             '經理人': '泛歐洲股票團隊',
             '計價幣別': '英鎊',
             '註冊地': '盧森堡'})

In [622]:
pd.DataFrame([getForeignFundInfo(soup)])

星星幾顆:3


,fundid,保管機構,傘狀基金,台灣總代理,基金名稱,基金英文名稱,基金規模,基金評等,基金類型,成立日期,投資區域,投資標的,投資策略,指標指數,海外發行公司,經理人,計價幣別,註冊地
0,ANZ11-A06,法國巴黎銀行盧森堡分行,Y,Aberdeen安本投顧,安本環球英國股票基金,Aberdeen Global-UK Equity Fund A2,37 百萬英鎊(2016/12/31),3,單一國家型基金,1988/04/26,英國,股票型,本基金的投資目標是通過將該項基金至少三分之二的資產，投資於在英國設有註冊辦事處的公司；及/ ...,FTSE ALL Share,Aberdeen安本資產管理,泛歐洲股票團隊,英鎊,盧森堡


### 海外持股狀況

In [593]:
url_wbTest_stock = 'http://mmafund.sinopac.com/w/wb/wb04.djhtm?a=ANZ45-A28'
html_text_wb = requests.get(url_wbTest_stock).text
soup = BS(html_text_wb,"lxml")

#### 最後一個table 
[網頁](http://mmafund.sinopac.com/w/wb/wb04.djhtm?a=ANZ45-A28)

In [440]:
list(zip(soup.select('.wfb2l'),soup.select('.wfb2r')))

[(<td class="wfb2l">TSMC 台積電,台灣</td>, <td class="wfb2r">3.80% </td>),
 (<td class="wfb2l">Astra International 阿斯特拉國際,印尼</td>,
  <td class="wfb2r">3.70% </td>),
 (<td class="wfb2l">AIA Group 友邦保險,中國/香港</td>, <td class="wfb2r">3.10% </td>),
 (<td class="wfb2l">ITC (消費必需品),印度</td>, <td class="wfb2r">2.80% </td>),
 (<td class="wfb2l">Grupo Financiero Banorte (金融),墨西哥</td>,
  <td class="wfb2r">2.60% </td>)]

In [441]:
list(zip(soup.select('.wfb5l'),soup.select('.wfb5r')))

[(<td class="wfb5l">Samsung Electronics 三星電子,南韓</td>,
  <td class="wfb5r">4.70% </td>),
 (<td class="wfb5l">Housing Dev Finance (金融),印度</td>,
  <td class="wfb5r">3.80% </td>),
 (<td class="wfb5l">China Mobile 中國移動,中國/香港</td>,
  <td class="wfb5r">3.30% </td>),
 (<td class="wfb5l">Fomento Economico Mexicano (消費必需品),墨西哥</td>,
  <td class="wfb5r">2.80% </td>),
 (<td class="wfb5l">Banco Bradesco-Adr (金融),巴西</td>,
  <td class="wfb5r">2.70% </td>)]

In [601]:
a = list(zip(soup.select('.wfb2l'),soup.select('.wfb2r')))
b = list(zip(soup.select('.wfb5l'),soup.select('.wfb5r')))
list(zip(*[(e1.text,e2.text.strip()) for (e1,e2) in a+b]))
# for e in test:
#     print(list(e))

[('TSMC 台積電,台灣',
  'Astra International 阿斯特拉國際,印尼',
  'AIA Group 友邦保險,中國/香港',
  'ITC (消費必需品),印度',
  'Grupo Financiero Banorte (金融),墨西哥',
  'Samsung Electronics 三星電子,南韓',
  'Housing Dev Finance (金融),印度',
  'China Mobile 中國移動,中國/香港',
  'Fomento Economico Mexicano (消費必需品),墨西哥',
  'Banco Bradesco-Adr (金融),巴西'),
 ('3.80%',
  '3.70%',
  '3.10%',
  '2.80%',
  '2.60%',
  '4.70%',
  '3.80%',
  '3.30%',
  '2.80%',
  '2.70%')]

In [520]:
update_date = '/'.join(re.findall(r"\d+",soup.select('.wfb1ar')[0].text))
update_date

'2016/12/31'

In [457]:
update_date = '/'.join(re.findall(r"\d+",soup.select('.wfb1ar')[-1].text)) ## 資料月份
print('資料月份:{}'.format(update_date))

資料月份:2016/12


In [614]:
def getForeignStockHolding(soup):
    
    """取得[境外]持股資料中股票張數(表格)
    params 
    ======
    input: soup --- html經BS包裝後變數
    return : defaultdict {持股名稱:[],
                            fundid:[],                            
                            比例:[],
                            資料月份,[]}
    
    """
    ### fundid ####
    fundid = re.findall(r"(?:a=)(.+)",soup.select('#itemTab')[0].find('a').get('href'))[0]
    
    ### 資料月份 ###    
    update_date = '/'.join(re.findall(r"\d+",soup.select('.wfb1ar')[-1].text)) ## 資料月份
    print('資料月份:{}'.format(update_date))
    
    a = list(zip(soup.select('.wfb2l'),soup.select('.wfb2r')))
    b = list(zip(soup.select('.wfb5l'),soup.select('.wfb5r')))
    dataList = zip(*[(e1.text,e2.text.strip()) for (e1,e2) in a+b]) # [(k1,k2,k3),(v1,v2,v3...)]
    
    colNames = ['持股名稱','比例']
    foreignStockDict = defaultdict(list)
    foreignStockDict['資料月份'] = update_date
    foreignStockDict['fundid'] = fundid
    for index,data in enumerate(dataList):
        foreignStockDict[colNames[index]] = list(data)
#     print(foreignStockDict)

    return foreignStockDict

In [615]:
foreign_stock = getForeignStockHolding(soup)

資料月份:2016/12


In [616]:
pd.DataFrame(foreign_stock)

,fundid,持股名稱,比例,資料月份
0,ANZ45-A28,"TSMC 台積電,台灣",3.80%,2016/12
1,ANZ45-A28,"Astra International 阿斯特拉國際,印尼",3.70%,2016/12
2,ANZ45-A28,"AIA Group 友邦保險,中國/香港",3.10%,2016/12
3,ANZ45-A28,"ITC (消費必需品),印度",2.80%,2016/12
4,ANZ45-A28,"Grupo Financiero Banorte (金融),墨西哥",2.60%,2016/12
5,ANZ45-A28,"Samsung Electronics 三星電子,南韓",4.70%,2016/12
6,ANZ45-A28,"Housing Dev Finance (金融),印度",3.80%,2016/12
7,ANZ45-A28,"China Mobile 中國移動,中國/香港",3.30%,2016/12
8,ANZ45-A28,"Fomento Economico Mexicano (消費必需品),墨西哥",2.80%,2016/12
9,ANZ45-A28,"Banco Bradesco-Adr (金融),巴西",2.70%,2016/12


#### 圓餅圖
[網頁](http://mmafund.sinopac.com/w/wb/wb04.djhtm?a=ANZ45-A28)

In [475]:
for index,titleText in enumerate(investTitle):
    titleList =re.findall(pat2,titleText)
    print(titleList)

['基金持股分佈-依區域[萬元]', '印度', '其他國家', '中國/香港', '巴西', '墨西哥', '南韓', '泰國', '印尼', '俄羅斯', '台灣', '南非', '現金']
['基金持股分佈-依產業[萬元]', '金融', '消費必需品', '資訊科技', '可選消費品', '原物料', '其他產業', '不動產', '能源', '現金']


In [487]:
re.findall(r"(?:a=)(.+)",soup.select('#itemTab')[0].find('a').get('href'))[0]


'ANZ45-A28'

In [594]:
def getForeignShareHolding(html_text):
    """取得國外持股資料(圓餅圖)
    剖析mma中html含有js程式碼，資料隱藏在js其中
    params
    html_text : raw text(str)
    return : list of defaultdict
    """
    def getShareHoldingTable(stockGroupList):
        """轉換國外持股圓餅圖資料(getForeignShareHolding)為dict格式(pd.dataframe可直接使用)    
        """
        stockGroup = defaultdict(list)
        for index,(k,v) in enumerate(stockGroupList):

            if index >1:
                stockGroup['項目'].append(k) 
                stockGroup['投資金額(美元:萬)'].append(v)
            else:
                stockGroup[k] = v
        return stockGroup
    
    soup = BS(html_text,"lxml")
    update_date = '/'.join(re.findall(r"\d+",soup.select('.wfb1ar')[0].text)) ## 資料更新日期
    
    ### fundid ####
    fundid = re.findall(r"(?:a=)(.+)",soup.select('#itemTab')[0].find('a').get('href'))[0]
    ###############
    
    string1 = 'DJGraphObj1' # 切出目標字串    
    target_text = html_text_wb[html_text_wb.index(string1):]

    pat1 = r"(?:\'Title\':)(.+)"
    investTitle = re.findall(pat1,target_text) # 取得並切分表單table

    pat2 = r"(?:\')(.*?)(?:\')" # 取出包含在 ' '內的字串
    pat3 = r"(?:\'PieV\':)(.+)" # 取出包含在 PieV 後的字串
    #     investTitleByStock = re.findall(pat2,investTitle[0]) ## 依產業標題(List)
    #     investTitleByStock
    table = defaultdict(list)
    tableAns = []
    
    for index,titleText in enumerate(investTitle):
        titleList = re.findall(pat2,titleText)
        if len(titleList) == 1:
            continue
        colname = titleList[0]
        titleList = titleList[1:]
        titleList.insert(0,'fundid')
        titleList.insert(1,'資料日期')
        valueList = re.findall(pat2,re.findall(pat3,target_text)[index])
        valueList.insert(0,fundid)
        valueList.insert(1,update_date)
    #         print(titleList,valueList)
        table[colname]= (list(zip(titleList,valueList)))
        tableAns.append(getShareHoldingTable(table[colname]))
        
    return tableAns

In [595]:
foreign_share = getForeignShareHolding(html_text_wb)

In [596]:
foreign_share[0]

defaultdict(list,
            {'fundid': 'ANZ45-A28',
             '投資金額(美元:萬)': ['97440',
              '91640',
              '89320',
              '70760',
              '42340',
              '41760',
              '29000',
              '28420',
              '27840',
              '27260',
              '26100',
              '8120'],
             '資料日期': '2016/12/31',
             '項目': ['印度',
              '其他國家',
              '中國/香港',
              '巴西',
              '墨西哥',
              '南韓',
              '泰國',
              '印尼',
              '俄羅斯',
              '台灣',
              '南非',
              '現金']})

In [527]:
pd.DataFrame(foreign_share[0])

,fundid,投資金額(美元:萬),項目
0,ANZ45-A28,97440,印度
1,ANZ45-A28,91640,其他國家
2,ANZ45-A28,89320,中國/香港
3,ANZ45-A28,70760,巴西
4,ANZ45-A28,42340,墨西哥
5,ANZ45-A28,41760,南韓
6,ANZ45-A28,29000,泰國
7,ANZ45-A28,28420,印尼
8,ANZ45-A28,27840,俄羅斯
9,ANZ45-A28,27260,台灣


In [541]:
pd.DataFrame(foreign_share[1])

,fundid,投資金額(美元:萬),資料日期,項目
0,ANZ45-A28,165300,2016/12/31,金融
1,ANZ45-A28,101500,2016/12/31,消費必需品
2,ANZ45-A28,73080,2016/12/31,資訊科技
3,ANZ45-A28,56260,2016/12/31,可選消費品
4,ANZ45-A28,52780,2016/12/31,原物料
5,ANZ45-A28,47560,2016/12/31,其他產業
6,ANZ45-A28,39440,2016/12/31,不動產
7,ANZ45-A28,35960,2016/12/31,能源
8,ANZ45-A28,8120,2016/12/31,現金


In [516]:
pd.DataFrame(tableAns[1])

,fundid,投資金額(美元:萬),項目
0,ANZ45-A28,165300,金融
1,ANZ45-A28,101500,消費必需品
2,ANZ45-A28,73080,資訊科技
3,ANZ45-A28,56260,可選消費品
4,ANZ45-A28,52780,原物料
5,ANZ45-A28,47560,其他產業
6,ANZ45-A28,39440,不動產
7,ANZ45-A28,35960,能源
8,ANZ45-A28,8120,現金


______

# 輔助函數

In [266]:
def getFundBasicInfo(soup):
    """
    取得基金基本資料
    
    params
    ======
    input  : soup
    output : defaultDict ==>{'fundid': 'ACUS03-UN2','主要投資區域': '台灣',
                            '保管銀行': '中國信託銀行','基金公司': '永豐投信',...}
    """
    #### fundid ####
    fundid = re.findall(r'(?:a=)(.+)',soup.select('#itemTab')[0].find('a').get('href'))[0]
    ################
    fieldSoup = soup.select('.wfb5l')
    fieldList = [fundid]
    for e in fieldSoup[:10]:
    #     print(e.text)
        fieldList.append(e.text)
    starNum = getFundStar(fieldSoup[-3])
    fieldList.append(starNum)
    fieldList.append(fieldSoup[-1].text)
    info_colName = [
        'fundid',
        '基金名稱',
        '基金公司',
        '成立日期',
        '基金經理人',
        '基金規模(億)',
        '成立時規模(億)',
        '基金類型',
        '保管銀行',
        '主要投資區域',
        '投資區域',
        '基金評等',
        '投資標的'
    ]
    dataList = list(zip(info_colName,fieldList)) ## list of tuples ==> [(fundid,xxxx),(基金名稱,xxxxx),(基金公司,xxxx),...]
    
    dataDict = defaultdict(list)
    ## 將資料格式轉換為 defaultdict ##
    for k,v in dataList:
        dataDict[k] = v
    return dataDict

In [273]:
temp = getFundBasicInfo(soup)
temp

星星幾顆:4.5


defaultdict(list,
            {'fundid': 'ACUS03-UN2',
             '主要投資區域': '台灣',
             '保管銀行': '中國信託銀行',
             '基金公司': '永豐投信',
             '基金名稱': '永豐領航科技基金',
             '基金經理人': '楊明昌',
             '基金規模(億)': '6.43台幣 (2017/01/31)',
             '基金評等': 4.5,
             '基金類型': '國內股票開放型科技類',
             '成立日期': '1998年9月4日',
             '成立時規模(億)': '2.00(台幣)',
             '投資區域': '台灣',
             '投資標的': '本基金以投資從事或轉投資於通訊、資訊、半導體、關鍵性零組件、生物工程、消費性電子、精密機械與自動化、航太、高級材料、特用化學與製藥、醫療保健、污染防治、資源開發、高級感測及其他經財政部會同主管機關認定重要科技之上市、上櫃股票為主，投資總額不低於淨資產之60%'})

In [303]:
def getFundManager(soup):
    """
    取得歷任經理人資料
    """
    #### fundid ####
    fundid = re.findall(r'(?:a=)(.+)',soup.select('#itemTab')[0].find('a').get('href'))[0]
    ################
    managerList = []
    managerDict = defaultdict(list)
    managerBSTable = soup.select('#oMainTable')[1]

    colNames = [        
        '經理人',
        '時間',
        '期間(月)',
        '操作績效(%)',
        '台股績效(%)',
        '現任基金'
    ]
#     managerList = []
    managerDict['fundid'] = fundid
    for index,manager in enumerate(managerBSTable.select('tr')[2:]):
        row = []
        for index,field in enumerate(manager.select('td')):
            try:
                field = float(field.text)
                row.append(field) # 轉換數字部分
            except ValueError as e:
                field = field.text
                row.append(field) 
            managerDict[colNames[index]].append(field) 
#         print(row)
    return managerDict

In [305]:
manager = getFundManager(soup)
# pd.DataFrame(manager)

defaultdict(list,
            {'fundid': 'ACUS03-UN2',
             '台股績效(%)': [3.41,
              1.21,
              5.97,
              10.82,
              -12.25,
              15.11,
              -15.3,
              108.27,
              -26.52,
              7.4,
              -16.56],
             '操作績效(%)': [4.21,
              27.23,
              7.63,
              46.81,
              -22.13,
              6.0,
              -27.62,
              201.6,
              -41.2,
              5.08,
              21.6],
             '時間': ['2017/2/3至今',
              '2014/08/01至2017/2/2',
              '2014/05/02至2014/7/31',
              '2012/04/01至2014/5/1',
              '2011/04/29至2012/3/31',
              '2010/04/20至2011/4/28',
              '2007/11/05至2010/4/19',
              '2003/01/02至2007/11/4',
              '2002/03/20至2003/1/1',
              '2001/03/01至2002/3/19',
              '1998/09/01至2001/2/28'],
             '期間(月)': [1.0,
              30.0,
    

In [5]:
def getFundStar(starTag):
    '''取得基金評等數字(影像<星星>標記 --> 數字)
    params ------
    starTag: 含有星號圖案的bs4 tag
    return: 基金評等(int)
    '''
    starList = starTag.select('img') # bs4 tag
    starList = [star['src'] for star in starList ] # 取出src裡面的url
    starStr = '**'.join(starList) # 接成string
    if re.findall('/BT_Coin_b.gif',starStr):
        starNum = 0.5
    else:
        starNum = 0
    starNum += len(re.findall('/BT_Coin_a.gif',starStr))
    print('星星幾顆:{}'.format(starNum))
    return starNum

In [212]:
def getDomesticShareHolding(html_text):
    """取得國內持股資料(圓餅圖)
    剖析mma中html含有js程式碼，資料隱藏在js其中
    params
    html_text : raw text(str)
    return : list of defaultdict
    """
    def getShareHoldingTable(stockGroupList):
        """轉換國內持股圓餅圖資料(getDomesticShareHolding)為dict格式(pd.dataframe可直接使用)    
        """
        stockGroup = defaultdict(list)
        for index,(k,v) in enumerate(stockGroupList):

            if index >1:
                stockGroup['項目'].append(k) 
                stockGroup['投資金額(萬元)'].append(v)
            else:
                stockGroup[k] = v
        return stockGroup

    ### 取得 fundid ###
    soup = BS(html_text,"lxml")
    fundid = re.findall(r"(?:a=)(.+)",soup.select('#itemTab')[0].find('a').get('href'))[0]
    print('fundid:{}'.format(fundid))
    ##### 取得 資料日期 ####
    update_dateStr = re.findall(r"\d+\/\d+\/\d+",soup.select('.wfb1ar')[-1].text)[0] # xx分布--資料日期
    #######################
    string1 = 'DJGraphObj1' # 切出目標字串    
    target_text = html_text[html_text.index(string1):]
    
    pat1 = r"(?:\'Title\':)(.+)"
    investTitle = re.findall(pat1,target_text) # 取得並切分表單table
           
    pat2 = r"(?:\')(.*?)(?:\')" # 取出包含在 ' '內的字串
    pat3 = r"(?:\'PieV\':)(.+)" # 取出包含在 PieV 後的字串
#     investTitleByStock = re.findall(pat2,investTitle[0]) ## 依產業標題(List)
#     investTitleByStock
    table = defaultdict(list)
    tableAns = []
    for index,titleText in enumerate(investTitle):
        titleList = re.findall(pat2,titleText)
        if len(titleList) == 1:
            continue
        colname = titleList[1]
        titleList = titleList[2:]
        titleList.insert(0,'fundid')
        titleList.insert(1,'資料日期')
        valueList = re.findall(pat2,re.findall(pat3,target_text)[index])
        valueList.insert(0,fundid)
        valueList.insert(1,update_dateStr)
#         print(titleList,valueList)
        table[colname]= (list(zip(titleList,valueList)))
        tableAns.append(getShareHoldingTable(table[colname]))
    return tableAns

In [213]:
getDomesticShareHolding(html_text)

fundid:ACSH03-170


C:\Anaconda2\envs\py3k\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda2\envs\py3k\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


[defaultdict(list,
             {'fundid': 'ACSH03-170',
              '投資金額(萬元)': ['18374',
               '12738',
               '10835',
               '8001',
               '6394',
               '4573',
               '3213',
               '2653',
               '3510',
               '1425',
               '1953',
               '1104',
               '7622'],
              '產業': ['上市半導體',
               '上櫃半導體',
               '上市光電',
               '上市電子零組件',
               '上櫃電子零組件',
               '上市其他',
               '上櫃其他電子',
               '上市其他電子',
               '上市電機機械',
               '上市通信網路',
               '上市建材營造',
               '上櫃通信網路',
               '其它'],
              '資料日期': '2017/02/17'})]

In [317]:
def getShareHoldingTable(stockGroupList):
    """轉換國內持股圓餅圖資料(getDomesticShareHolding)為dict格式(pd.dataframe可直接使用)    
    """
    stockGroup = defaultdict(list)
    for index,(k,v) in enumerate(stockGroupList):

        if index >1:
            stockGroup['項目'].append(k) 
            stockGroup['投資金額(萬元)'].append(v)
        else:
            stockGroup[k] = v
    return stockGroup

In [318]:
def getDomesticStockHolding(soup):
    """取得國內持股資料中股票張數(表格)
    params 
    ======
    input: soup --- html經BS包裝後變數
    return : defaultdict {增減:[],
                            股票名稱:[],
                            持股(千股):[],
                            比例:[],
                            資料月份,[]}
    """
    ### 取得 fundid ###
    fundid = re.findall(r"(?:a=)(.+)",soup.select('#itemTab')[0].find('a').get('href'))[0]
    ###############
    
    def parseElement(e):
        """利用此函數剖析表格內元素
        """
        row1 = []; row2=[];
        for index,field in enumerate(e.select('td')):        
            if index > 3:
                row2.append(field.text)
            else:
                row1.append(field.text)
        return row1,row2
    
    
    ######## 資料月份 ##########
    update_dateStr = soup.select('#oMainTable')[1].select('tr')[0].text # 資料月份
    update_date = re.findall(r'\d+/\d+',update_dateStr)[0]
    date_colname = '資料月份'
    print(date_colname,update_date)
    
    ###########################
    
    
    fundShareTableList = []
    rowData = {}
    for index,e in enumerate(soup.select('#oMainTable')[1].select('tr')[1:]):
        colnames = []
        if index == 0:
            col1,col2 = parseElement(e)
            col1+=['fundid',date_colname]
            col2+=['fundid',date_colname]
        else:            
            row1,row2 = parseElement(e)
            row1+=[fundid,update_date]
            row2+=[fundid,update_date]
            
            fundShareTableList.append(list(zip(col1,row1))) 
            fundShareTableList.append(list(zip(col2,row2)))
    ### 輸出成dict格式 ##
    stocks_dict = defaultdict(list)
    for row in fundShareTableList:
        for name,value in row:    
            stocks_dict[name].append(value)
    return stocks_dict

In [319]:
tables = getDomesticStockHolding(soup)
tables

資料月份 2017/01


defaultdict(list,
            {'fundid': ['ACJS24-Z21',
              'ACJS24-Z21',
              'ACJS24-Z21',
              'ACJS24-Z21',
              'ACJS24-Z21',
              'ACJS24-Z21',
              'ACJS24-Z21',
              'ACJS24-Z21',
              'ACJS24-Z21',
              'ACJS24-Z21'],
             '增減': ['N/A',
              '-0.14%',
              '4.36%',
              '0.00%',
              '0.05%',
              '-0.15%',
              '0.14%',
              '0.89%',
              '-3.25%',
              '0.03%'],
             '持股(千股)': ['N/A',
              'N/A',
              '6',
              'N/A',
              '130',
              'N/A',
              'N/A',
              '145',
              '90',
              'N/A'],
             '比例': ['6.65',
              '3.07',
              '6.61',
              '2.36',
              '5.97',
              '2.25',
              '3.63',
              '2.24',
              '3.19',
              '2.23'],
        

In [320]:
pd.DataFrame(tables)

,fundid,增減,持股(千股),比例,股票名稱,資料月份
0,ACJS24-Z21,N/A,N/A,6.65,Tencent,2017/01
1,ACJS24-Z21,-0.14%,N/A,3.07,KOITO MFG CO,2017/01
2,ACJS24-Z21,4.36%,6,6.61,大立光,2017/01
3,ACJS24-Z21,0.00%,N/A,2.36,華夏滬深300指數ETF,2017/01
4,ACJS24-Z21,0.05%,130,5.97,台積電,2017/01
5,ACJS24-Z21,-0.15%,N/A,2.25,Telekomunikasi Indonesia TBK,2017/01
6,ACJS24-Z21,0.14%,N/A,3.63,NIDEC CORP,2017/01
7,ACJS24-Z21,0.89%,145,2.24,正新,2017/01
8,ACJS24-Z21,-3.25%,90,3.19,環球晶,2017/01
9,ACJS24-Z21,0.03%,N/A,2.23,iShares 安碩富時A50 中國指數ETF,2017/01


In [321]:
list(zip(*tables[0]))    

[]

In [322]:
url = 'http://mmafund.sinopac.com/w/wr/wr04.djhtm?a=ACJS24-Z21'
url_na = 'http://mmafund.sinopac.com/w/wr/wr04.djhtm?a=ACGC40-270'
res = requests.get(url)
html_text = res.text
soup = BS(html_text,"lxml")

C:\Anaconda2\envs\py3k\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda2\envs\py3k\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [323]:
ACJS24_Z21 = getDomesticShareHolding(html_text)

fundid:ACJS24-Z21


C:\Anaconda2\envs\py3k\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda2\envs\py3k\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [335]:
pd.DataFrame(ACJS24_Z21[0])

,fundid,投資金額(萬元),資料日期,項目
0,ACJS24-Z21,16416,2017/01/31,亞洲不含日本
1,ACJS24-Z21,4583,2017/01/31,日本
2,ACJS24-Z21,3558,2017/01/31,大陸地區
3,ACJS24-Z21,2681,2017/01/31,上市光電
4,ACJS24-Z21,2431,2017/01/31,上市半導體
5,ACJS24-Z21,1619,2017/01/31,上櫃半導體
6,ACJS24-Z21,937,2017/01/31,上市橡膠
7,ACJS24-Z21,892,2017/01/31,上市電子零組件
8,ACJS24-Z21,844,2017/01/31,上市油電燃氣
9,ACJS24-Z21,440,2017/01/31,上櫃其他電子


In [325]:
pd.DataFrame(ACJS24_Z21[1])

,fundid,投資金額(萬元),資料日期,項目
0,ACJS24-Z21,10823,2017/01/31,香港
1,ACJS24-Z21,9652,2017/01/31,台灣
2,ACJS24-Z21,6179,2017/01/31,中國
3,ACJS24-Z21,4684,2017/01/31,日本
4,ACJS24-Z21,4402,2017/01/31,現金
5,ACJS24-Z21,4321,2017/01/31,東南亞
6,ACJS24-Z21,323,2017/01/31,韓國


In [326]:
pd.DataFrame(ACJS24_Z21[2])

,fundid,投資金額(萬元),資料日期,項目
0,ACJS24-Z21,13044,2017/01/31,科技
1,ACJS24-Z21,6502,2017/01/31,工業
2,ACJS24-Z21,5048,2017/01/31,消費
3,ACJS24-Z21,4765,2017/01/31,金融保險
4,ACJS24-Z21,4402,2017/01/31,現金
5,ACJS24-Z21,1858,2017/01/31,A股ETF
6,ACJS24-Z21,1696,2017/01/31,能源
7,ACJS24-Z21,1656,2017/01/31,原物料
8,ACJS24-Z21,848,2017/01/31,公共事業
9,ACJS24-Z21,525,2017/01/31,醫療保健


_______

## 資料庫

In [330]:
import pypyodbc

In [331]:
con = pypyodbc.connect("DRIVER={SQL Server};SERVER=xxxxx;UID=xxxxxx;PWD=xxxxxxxx;DATABASE=xxxxxxx")
cursor = con.cursor()

In [371]:
sql_insert = """
INSERT INTO [基金持股狀況_依持有類股分] 
VALUES (?,?,?,?)
"""


In [357]:
df = pd.DataFrame(dataToDB_dict)

In [343]:
dataToDB_dict = ACJS24_Z21[0]
dataToDB_pd_dict = pd.DataFrame(dataToDB_dict).to_dict()


In [350]:
dataToDB_pd_dict.keys()
    

dict_keys(['fundid', '投資金額(萬元)', '資料日期', '項目'])

In [377]:
for e in list(df.T.to_dict().values()):
#     cursor.execute(sql_insert,list(e.values()))
    print(list(e.values()))

dict_values(['ACJS24-Z21', '16416', '2017/01/31', '亞洲不含日本'])
dict_values(['ACJS24-Z21', '4583', '2017/01/31', '日本'])
dict_values(['ACJS24-Z21', '3558', '2017/01/31', '大陸地區'])
dict_values(['ACJS24-Z21', '2681', '2017/01/31', '上市光電'])
dict_values(['ACJS24-Z21', '2431', '2017/01/31', '上市半導體'])
dict_values(['ACJS24-Z21', '1619', '2017/01/31', '上櫃半導體'])
dict_values(['ACJS24-Z21', '937', '2017/01/31', '上市橡膠'])
dict_values(['ACJS24-Z21', '892', '2017/01/31', '上市電子零組件'])
dict_values(['ACJS24-Z21', '844', '2017/01/31', '上市油電燃氣'])
dict_values(['ACJS24-Z21', '440', '2017/01/31', '上櫃其他電子'])
dict_values(['ACJS24-Z21', '279', '2017/01/31', '上市電腦及週邊設備'])
dict_values(['ACJS24-Z21', '5702', '2017/01/31', '其它'])


In [375]:
con.commit()

In [379]:
for e in df.T.to_dict().values():
    print(list(e.values()))


['ACJS24-Z21', '16416', '2017/01/31', '亞洲不含日本']
['ACJS24-Z21', '4583', '2017/01/31', '日本']
['ACJS24-Z21', '3558', '2017/01/31', '大陸地區']
['ACJS24-Z21', '2681', '2017/01/31', '上市光電']
['ACJS24-Z21', '2431', '2017/01/31', '上市半導體']
['ACJS24-Z21', '1619', '2017/01/31', '上櫃半導體']
['ACJS24-Z21', '937', '2017/01/31', '上市橡膠']
['ACJS24-Z21', '892', '2017/01/31', '上市電子零組件']
['ACJS24-Z21', '844', '2017/01/31', '上市油電燃氣']
['ACJS24-Z21', '440', '2017/01/31', '上櫃其他電子']
['ACJS24-Z21', '279', '2017/01/31', '上市電腦及週邊設備']
['ACJS24-Z21', '5702', '2017/01/31', '其它']
